# 要約 
このJupyter Notebookでは、Kaggleの「LMSYS - Chatbot Arena」コンペティションにおいて、人間による好みの予測に挑戦しています。具体的には、異なる大規模言語モデル（LLM）が生成した応答の中から、どちらがユーザーに好まれるかを予測するための機械学習モデルを構築しています。

### 問題に対するアプローチ
Notebookは以下の主要なステップで構成されています：

1. **ライブラリのインポート**: pandas、numpy、nltkなどのデータ処理および自然言語処理ライブラリに加えて、XGBoost、LightGBM、CatBoostなどの機械学習ライブラリをインポートしています。

2. **データの読み込みおよび前処理**:
   - トレーニングデータとテストデータをCSVファイルから読み込み、不要な文字を取り除くなどの前処理を行います。
   - トークン化や特徴量の生成を行う`Preprocessor`クラスが定義され、コサイン類似度やジャッカード類似度の計算、n-gramの生成などが実装されています。

3. **特徴選択**: ANOVA F値に基づいて、最も有用な特徴量を25個選択します。

4. **モデルのトレーニングと評価**:
   - RandomForest、GradientBoosting、SVM、XGBoost、CatBoost、VotingClassifierなど、複数の機械学習モデルを使用します。
   - クロスバリデーションによってモデルのパフォーマンスを評価し、平均CVログロスを計算します。

5. **結果の報告**: 各モデルのCVログロスを比較し、最良のモデルを特定するとともに、特徴の重要度を示すDataFrameを生成します。

6. **提出ファイルの作成**: 最後に、テストデータに対する予測結果を用いて、提出用のCSVファイル（`submission.csv`）を出力します。

### 使用されている手法とライブラリ
- **データ処理**: pandas、numpy、nltk
- **特徴量生成**: コサイン類似度、ジャッカード類似度、n-gram、引用数カウント、トークン化などの手法を用いた特徴量の生成。
- **モデル**: RandomForest、GradientBoosting、SVM、XGBoost、CatBoost、VotingClassifier
- **評価手法**: StratifiedKFoldによるクロスバリデーション、log_lossによる評価

このNotebookは、テキストデータを用いた機械学習タスクに対する徹底した前処理、特徴量エンジニアリング、モデル評価のプロセスを示しており、選好予測の向上を目指しています。

---


# 用語概説 
以下は、Jupyter Notebookに含まれる専門用語の解説です。初心者向けに必要な域外の用語や少しマイナーなものに焦点を当てています。

1. **ガーベジコレクション(GC)**:
   自動的にメモリを管理し、不要なオブジェクトを削除する機能。メモリリークを避けるのに役立つ。`gc`モジュールを使用して、明示的にガーベジコレクションを起動できる。

2. **TF-IDF (Term Frequency-Inverse Document Frequency)**:
   文書内における単語の重要度を測るための指標。頻繁に出現する単語はあまり重要性がないとみなされ、逆に文書全体に広まっていない単語は重要と評価される。

3. **コサイン類似度 (Cosine Similarity)**:
   2つのベクトル間の角度を測定する指標で、特にテキストの類似度を測るためによく使われる。数値的には、内積をベクトルのノルムで割ったものと定義される。

4. **ジャッカード類似度 (Jaccard Similarity)**:
   2つの集合の類似度を測る方法。交差部分のサイズを合併部分のサイズで割ることで計算される。特に、テキスト間の重複した単語の割合を知るのに役立つ。

5. **n-gram**:
   テキストを「n」個の連続した項目に分けたもの。たとえば、バイグラムは2つの単語の組み合わせ、トライグラムは3つの単語の組み合わせとなる。

6. **トークン化(Tokenization)**:
   テキストデータを個々の単語やフレーズに分解するプロセス。自然言語処理において基本的なステップ。

7. **層化K分割交差検証 (Stratified K-Fold Cross-Validation)**:
   データセットをK個の部分に分ける方法で、各部分がターゲット変数の分布を保つようになっている。これにより、モデル評価がより信頼性の高いものになる。

8. **ANOVA F値 (Analysis of Variance F-Value)**:
   異なるグループ間の平均の違いを測る統計的方法。特徴選択で、各特徴がターゲット変数に対してどれだけ情報を持っているかを評価する。

9. **アーリーストップ(Early Stopping)**:
   モデルのトレーニング中に検証データの性能が向上しなくなった段階でトレーニングを早期に終了する仕組み。過学習を防ぐために有効。

これらの用語は、機械学習や自然言語処理の領域でよく使われるため、知識を深めるうえで重要です。

---


<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# 📚 Importing Libraries

</div>
<div class="column-right">

# 日本語訳

# 📚 ライブラリのインポート


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import gc
import os
import re
import numpy as np
import pandas as pd

import nltk
from nltk.util import ngrams
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb
import lightgbm as lgb
import catboost as cb
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.impute import SimpleImputer
```

</div>
<div class="column-right">

# 日本語訳

```python
import gc
import os
import re
import numpy as np
import pandas as pd

import nltk
from nltk.util import ngrams
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb
import lightgbm as lgb
import catboost as cb
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.impute import SimpleImputer

# gcモジュール: Pythonのガーベジコレクションを提供
# osモジュール: オペレーティングシステムの機能を利用
# reモジュール: 正規表現による文字列操作
# numpy: 数値計算用のライブラリ
# pandas: データ操作用のライブラリ
# nltk: 自然言語処理用のライブラリ
# matplotlib.pyplot: データ可視化用の2Dプロットライブラリ
# seaborn: より洗練されたデータ可視化用のライブラリ
# xgboost: 勾配ブースティングアルゴリズムの実装
# lightgbm: LightGBMアルゴリズムの実装
# catboost: CatBoostアルゴリズムの実装
# sklearn: 機械学習用の多数のツールを提供
```

</div>
</details>

In [ ]:
import gc
import os
import re
import numpy as np
import pandas as pd

import nltk
from nltk.util import ngrams
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb
import lightgbm as lgb
import catboost as cb
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.impute import SimpleImputer

# gcモジュール: Pythonのガーベジコレクションを提供
# osモジュール: オペレーティングシステムの機能を利用
# reモジュール: 正規表現による文字列操作
# numpy: 数値計算用のライブラリ
# pandas: データ操作用のライブラリ
# nltk: 自然言語処理用のライブラリ
# matplotlib.pyplot: データ可視化用の2Dプロットライブラリ
# seaborn: より洗練されたデータ可視化用のライブラリ
# xgboost: 勾配ブースティングアルゴリズムの実装
# lightgbm: LightGBMアルゴリズムの実装
# catboost: CatBoostアルゴリズムの実装
# sklearn: 機械学習用の多数のツールを提供

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# ⚙️ Configuration Class

</div>
<div class="column-right">

# 日本語訳

# ⚙️ 設定クラス


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
class config:
    root = "/kaggle/input/lmsys-chatbot-arena/"
    train_path = os.path.join(root, "train.csv")
    test_path = os.path.join(root, "test.csv")
    sample_submission_path = os.path.join(root, "sample_submission.csv")
    seed = 42
    n_splits = 10
```

</div>
<div class="column-right">

# 日本語訳

```python
class config:
    root = "/kaggle/input/lmsys-chatbot-arena/"
    train_path = os.path.join(root, "train.csv")
    test_path = os.path.join(root, "test.csv")
    sample_submission_path = os.path.join(root, "sample_submission.csv")
    seed = 42
    n_splits = 10
```

</div>
</details>

In [ ]:
class config:
    root = "/kaggle/input/lmsys-chatbot-arena/"
    train_path = os.path.join(root, "train.csv")
    test_path = os.path.join(root, "test.csv")
    sample_submission_path = os.path.join(root, "sample_submission.csv")
    seed = 42
    n_splits = 10

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original


# 📊 Loading and Processing Data

we will load our training and test datasets, and apply some preprocessing. This includes:


1. **Loading Data**: Read the CSV files into pandas DataFrames.
2. **Subsampling**: If the test dataset has less than 10 rows, subsample the training dataset to 10,000 rows for quicker processing.
3. **Processing Strings**: Clean and process the string columns (`prompt`, `response_a`, `response_b`) by removing unwanted characters.
4. **Shape and Missing Values**: Print the shape of the datasets and count missing values to understand the data structure and quality.

</div>
<div class="column-right">

# 日本語訳


# 📊 データの読み込みと処理

トレーニングデータセットとテストデータセットを読み込み、一部の前処理を適用します。これには以下が含まれます:

1. **データの読み込み**: CSVファイルをpandasのDataFrameに読み込みます。
2. **サブサンプリング**: テストデータセットの行数が10未満の場合は、トレーニングデータセットから10,000行サブサンプリングして迅速に処理します。
3. **文字列の処理**: 不要な文字を削除して、文字列列（`prompt`, `response_a`, `response_b`）をクリーンアップし処理します。
4. **データの形状と欠損値**: データセットの形状を出力し、欠損値をカウントしてデータ構造と品質を理解します。


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
train = pd.read_csv(config.train_path)
test = pd.read_csv(config.test_path)
sample_submission = pd.read_csv(config.sample_submission_path)

if test.shape[0] < 10:
    train = train.iloc[:10000]
    
def process(input_str):
    stripped_str = input_str.strip('[]')
    sentences = [s.strip('"') for s in stripped_str.split('","')]
    return  ' '.join(sentences)

train["prompt"] = train["prompt"].apply(process)
train["response_a"] = train["response_a"].apply(process)
train["response_b"] = train["response_b"].apply(process)

test["prompt"] = test["prompt"].apply(process)
test["response_a"] = test["response_a"].apply(process)
test["response_b"] = test["response_b"].apply(process)

print(f"train shape: {train.shape}")
print(f"test shape: {test.shape}")
print("-"*90)
print(f"train missing values: {train.isnull().sum().sum()}")
print(f"test missing values: {test.isnull().sum().sum()}")
print("-"*90)

train.head()
```

</div>
<div class="column-right">

# 日本語訳

```python
train = pd.read_csv(config.train_path)  # 訓練データを読み込む
test = pd.read_csv(config.test_path)    # テストデータを読み込む
sample_submission = pd.read_csv(config.sample_submission_path)  # 提出用のサンプルデータを読み込む

# テストデータの行数が10未満であれば、トレーニングデータの最初の10,000行を使用する
if test.shape[0] < 10:
    train = train.iloc[:10000]
    
def process(input_str):
    stripped_str = input_str.strip('[]')  # 文字列の前後のブラケットを削除
    sentences = [s.strip('"') for s in stripped_str.split('","')]  # 文字列を分割してクォーテーションを削除
    return  ' '.join(sentences)  # 結果をスペースで繋げて返す

# 各列にprocess関数を適用して文字列をクリーンアップ
train["prompt"] = train["prompt"].apply(process)
train["response_a"] = train["response_a"].apply(process)
train["response_b"] = train["response_b"].apply(process)

test["prompt"] = test["prompt"].apply(process)
test["response_a"] = test["response_a"].apply(process)
test["response_b"] = test["response_b"].apply(process)

# データセットの形状を出力
print(f"train shape: {train.shape}")
print(f"test shape: {test.shape}")
print("-"*90)
# 欠損値の計算
print(f"train missing values: {train.isnull().sum().sum()}")
print(f"test missing values: {test.isnull().sum().sum()}")
print("-"*90)

train.head()  # トレーニングデータの最初の5行を表示
```

</div>
</details>

In [ ]:
train = pd.read_csv(config.train_path)  # 訓練データを読み込む
test = pd.read_csv(config.test_path)    # テストデータを読み込む
sample_submission = pd.read_csv(config.sample_submission_path)  # 提出用のサンプルデータを読み込む

# テストデータの行数が10未満であれば、トレーニングデータの最初の10,000行を使用する
if test.shape[0] < 10:
    train = train.iloc[:10000]
    
def process(input_str):
    stripped_str = input_str.strip('[]')  # 文字列の前後のブラケットを削除
    sentences = [s.strip('"') for s in stripped_str.split('","')]  # 文字列を分割してクォーテーションを削除
    return  ' '.join(sentences)  # 結果をスペースで繋げて返す

# 各列にprocess関数を適用して文字列をクリーンアップ
train["prompt"] = train["prompt"].apply(process)
train["response_a"] = train["response_a"].apply(process)
train["response_b"] = train["response_b"].apply(process)

test["prompt"] = test["prompt"].apply(process)
test["response_a"] = test["response_a"].apply(process)
test["response_b"] = test["response_b"].apply(process)

# データセットの形状を出力
print(f"train shape: {train.shape}")
print(f"test shape: {test.shape}")
print("-"*90)
# 欠損値の計算
print(f"train missing values: {train.isnull().sum().sum()}")
print(f"test missing values: {test.isnull().sum().sum()}")
print("-"*90)

train.head()  # トレーニングデータの最初の5行を表示

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# 🛠️ Preprocessing Class Definition

This class, `Preprocessor`, contains several methods to process and feature-engineer the text data. Here’s a breakdown of its functionalities:

#### Cosine Similarity
- **Formula**: 
  $$\text{cosine_similarity} = \frac{A \cdot B}{\|A\| \|B\|}$$

- **Description**: Cosine similarity measures the cosine of the angle between two vectors, providing a metric of how similar the texts are.

#### Jaccard Similarity
- **Formula**:

  $$\text{Jaccard_similarity} = \frac{|A \cap B|}{|A \cup B|}$$

- **Description**: Jaccard similarity measures the similarity between two sets by comparing their intersection and union.

#### Count Quotes
- **Description**: This method identifies and counts both single and double quoted texts in a string, giving an idea of how many quotations are present.

#### Tokenize
- **Description**: This method splits the text into individual words (tokens), which can be used for further analysis like generating n-grams or calculating overlaps.

#### Generate N-grams
- **Description**: N-grams are contiguous sequences of 'n' items from a given text. This method helps in analyzing the text at different levels of granularity (unigrams, bigrams, trigrams, etc.).

#### Count N-gram Overlaps
- **Description**: This method calculates how many n-grams are common between two texts, helping to measure their similarity.

#### Run
- **Description**: This method processes the entire dataset, generating new features based on the above calculations, which can be used for training machine learning models.


$\frac{n!}{k!(n-k)!} = \binom{n}{k}$


</div>
<div class="column-right">

# 日本語訳

# 🛠️ 前処理クラス定義

`Preprocessor`クラスは、テキストデータを処理し特徴量エンジニアリングを行ういくつかのメソッドを含んでいます。以下にその機能の説明を示します：

#### コサイン類似度
- **数式**: 
  $$\text{cosine_similarity} = \frac{A \cdot B}{\|A\| \|B\|}$$

- **説明**: コサイン類似度は2つのベクトルの間の角度のコサインを測定し、テキストの類似度の指標を提供します。

#### ジャッカード類似度
- **数式**:

  $$\text{Jaccard_similarity} = \frac{|A \cap B|}{|A \cup B|}$$

- **説明**: ジャッカード類似度は、2つの集合の交差部分と合併部分を比較することで類似度を測定します。

#### 引用のカウント
- **説明**: このメソッドは、文字列内の単一および二重引用されたテキストを特定し、カウントすることで、引用の数を把握します。

#### トークン化
- **説明**: このメソッドは、テキストを個々の単語（トークン）に分割し、n-gramを生成したり、重複を計算するためのさらなる分析に使用できます。

#### N-gramの生成
- **説明**: N-gramは、与えられたテキストからの'n'項目の連続したシーケンスです。このメソッドは、異なる粒度（単語、2-gram、3-gramなど）でテキストを分析するのに役立ちます。

#### N-gramの重複カウント
- **説明**: このメソッドは、2つのテキスト間の共通のn-gramの数を計算し、類似度を測定するのに役立ちます。

#### 実行
- **説明**: このメソッドは、上記の計算に基づいて新しい特徴量を生成し、機械学習モデルのトレーニングに使用できる全データセットを処理します。

$\frac{n!}{k!(n-k)!} = \binom{n}{k}$



</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
class Preprocessor:

    def cosine_sim(self, text1: str, text2: str):
        try:
            vectorizer = TfidfVectorizer().fit_transform([text1, text2])
            vectors = vectorizer.toarray()
            cos_sim = cosine_similarity(vectors)
            return cos_sim[0][1]
        except:
            return np.nan

    def jaccard_sim(self, text1: str, text2: str):
        set1 = set(text1.split())
        set2 = set(text2.split())
        intersection = set1.intersection(set2)
        union = set1.union(set2)
        return len(intersection) / len(union)
    
    def count_quotes(self, text: str) -> int:
        single_quote_pattern = r"'(.*?)'"
        double_quote_pattern = r'"(.*?)"'
        single_quotes = re.findall(single_quote_pattern, text)
        double_quotes = re.findall(double_quote_pattern, text)
        total_quotes = len(single_quotes) + len(double_quotes)
        return len(single_quotes) + len(double_quotes)

    def tokenize(self, text: str):
        return nltk.word_tokenize(text.lower())

    def generate_ngrams(self, text: str, n: int):
        tokens = self.tokenize(text)
        return list(ngrams(tokens, n))

    def count_ngram_overlaps(self, text1: str, text2: str, n: int) -> int:
        try:
            ngrams1 = self.generate_ngrams(text1, n)
            ngrams2 = self.generate_ngrams(text2, n)
            counter1 = Counter(ngrams1)
            counter2 = Counter(ngrams2)
            overlap = counter1 & counter2
            overlap_count = sum(overlap.values())
            return overlap_count
        except:
            return 0
        
    def run(self, data: pd.DataFrame) -> pd.DataFrame:
        
        data["respa_respb_overlap_unigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["response_b"], 1), axis=1)
        data["respa_respb_overlap_bigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["response_b"], 2), axis=1)
        data["respa_respb_overlap_trigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["response_b"], 3), axis=1)

        data["respa_prompt_overlap_unigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["prompt"], 1), axis=1)
        data["respa_prompt_overlap_bigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["prompt"], 2), axis=1)
        data["respa_prompt_overlap_trigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["prompt"], 3), axis=1)

        data["respb_prompt_overlap_unigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_b"], x["prompt"], 1), axis=1)
        data["respb_prompt_overlap_bigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_b"], x["prompt"], 2), axis=1)
        data["respb_prompt_overlap_trigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_b"], x["prompt"], 3), axis=1)
        
        data["respa_len"] = data["response_a"].apply(lambda x: len(self.tokenize(x)))
        data["respb_len"] = data["response_b"].apply(lambda x: len(self.tokenize(x)))
        data["prompt_len"] = data["prompt"].apply(lambda x: len(self.tokenize(x)))
        
        data["respa_prompt_len_ratio"] = data["respa_len"] / data["prompt_len"]
        data["respb_prompt_len_ratio"] = data["respb_len"] / data["prompt_len"]
        data["respa_respb_len_ratio"] = data["respa_len"] / data["respb_len"]
        
        data["respa_respb_len_diff"] = data["respa_len"] - data["respb_len"]
        data["respa_prompt_len_diff"] = data["respa_len"] - data["prompt_len"]
        data["respb_prompt_len_diff"] = data["respb_len"] - data["prompt_len"]
        
        data["respa_prompt_overlap_unigram_ratio"] = data["respa_prompt_overlap_unigram"] / data["prompt_len"]
        data["respa_prompt_overlap_bigram_ratio"] = data["respa_prompt_overlap_bigram"] / data["prompt_len"]
        data["respa_prompt_overlap_trigram_ratio"] = data["respa_prompt_overlap_trigram"] / data["prompt_len"]

        data["respb_prompt_overlap_unigram_ratio"] = data["respb_prompt_overlap_unigram"] / data["prompt_len"]
        data["respb_prompt_overlap_bigram_ratio"] = data["respb_prompt_overlap_bigram"] / data["prompt_len"]
        data["respb_prompt_overlap_trigram_ratio"] = data["respb_prompt_overlap_trigram"] / data["prompt_len"]
        
        data["respa_quotes"] = data["response_a"].apply(lambda x: self.count_quotes(x))
        data["respb_quotes"] = data["response_b"].apply(lambda x: self.count_quotes(x))
        data["prompt_quotes"] = data["prompt"].apply(lambda x: self.count_quotes(x))
        
        data["respa_respb_cosine_sim"] = data.apply(lambda x: self.cosine_sim(x["response_a"], x["response_b"]), axis=1)
        data["respa_respb_jaccard_sim"] = data.apply(lambda x: self.jaccard_sim(x["response_a"], x["response_b"]), axis=1)
        
        data["respa_prompt_cosine_sim"] = data.apply(lambda x: self.cosine_sim(x["response_a"], x["prompt"]), axis=1)
        data["respa_prompt_jaccard_sim"] = data.apply(lambda x: self.jaccard_sim(x["response_a"], x["prompt"]), axis=1)
        
        data["respb_prompt_cosine_sim"] = data.apply(lambda x: self.cosine_sim(x["response_b"], x["prompt"]), axis=1)
        data["respb_prompt_jaccard_sim"] = data.apply(lambda x: self.jaccard_sim(x["response_b"], x["prompt"]), axis=1)
        
        return data
```

</div>
<div class="column-right">

# 日本語訳

```python
class Preprocessor:

    def cosine_sim(self, text1: str, text2: str):
        try:
            vectorizer = TfidfVectorizer().fit_transform([text1, text2])  # テキストのTF-IDFベクトルを作成
            vectors = vectorizer.toarray()  # ベクトルを配列に変換
            cos_sim = cosine_similarity(vectors)  # コサイン類似度を計算
            return cos_sim[0][1]  # コサイン類似度の結果を返す
        except:
            return np.nan  # エラーが発生した場合はNaNを返す

    def jaccard_sim(self, text1: str, text2: str):
        set1 = set(text1.split())  # テキスト1を単語の集合に変換
        set2 = set(text2.split())  # テキスト2を単語の集合に変換
        intersection = set1.intersection(set2)  # 交差部分を計算
        union = set1.union(set2)  # 合併部分を計算
        return len(intersection) / len(union)  # ジャッカード類似度を返す
    
    def count_quotes(self, text: str) -> int:
        single_quote_pattern = r"'(.*?)'"  # 単一引用のパターン
        double_quote_pattern = r'"(.*?)"'  # 二重引用のパターン
        single_quotes = re.findall(single_quote_pattern, text)  # 単一引用を抽出
        double_quotes = re.findall(double_quote_pattern, text)  # 二重引用を抽出
        total_quotes = len(single_quotes) + len(double_quotes)  # 合計の引用数
        return total_quotes  # 合計の引用数を返す

    def tokenize(self, text: str):
        return nltk.word_tokenize(text.lower())  # テキストを小文字にしトークン化

    def generate_ngrams(self, text: str, n: int):
        tokens = self.tokenize(text)  # トークン化されたテキストを取得
        return list(ngrams(tokens, n))  # n-gramを生成

    def count_ngram_overlaps(self, text1: str, text2: str, n: int) -> int:
        try:
            ngrams1 = self.generate_ngrams(text1, n)  # テキスト1からn-gramを生成
            ngrams2 = self.generate_ngrams(text2, n)  # テキスト2からn-gramを生成
            counter1 = Counter(ngrams1)  # テキスト1のn-gramのカウント
            counter2 = Counter(ngrams2)  # テキスト2のn-gramのカウント
            overlap = counter1 & counter2  # 共通のn-gramを計算
            overlap_count = sum(overlap.values())  # 重複の合計を計算
            return overlap_count  # 重複数を返す
        except:
            return 0  # エラーが発生した場合は0を返す
        
    def run(self, data: pd.DataFrame) -> pd.DataFrame:
        
        # それぞれの応答間でのn-gramオーバーラップを計算
        data["respa_respb_overlap_unigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["response_b"], 1), axis=1)
        data["respa_respb_overlap_bigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["response_b"], 2), axis=1)
        data["respa_respb_overlap_trigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["response_b"], 3), axis=1)

        # 各応答とプロンプト間でのn-gramオーバーラップを計算
        data["respa_prompt_overlap_unigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["prompt"], 1), axis=1)
        data["respa_prompt_overlap_bigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["prompt"], 2), axis=1)
        data["respa_prompt_overlap_trigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["prompt"], 3), axis=1)

        data["respb_prompt_overlap_unigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_b"], x["prompt"], 1), axis=1)
        data["respb_prompt_overlap_bigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_b"], x["prompt"], 2), axis=1)
        data["respb_prompt_overlap_trigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_b"], x["prompt"], 3), axis=1)
        
        # 各応答とプロンプトの長さを計算
        data["respa_len"] = data["response_a"].apply(lambda x: len(self.tokenize(x)))  # 応答Aの長さ
        data["respb_len"] = data["response_b"].apply(lambda x: len(self.tokenize(x)))  # 応答Bの長さ
        data["prompt_len"] = data["prompt"].apply(lambda x: len(self.tokenize(x)))  # プロンプトの長さ
        
        # 異なる長さの比率や差を計算
        data["respa_prompt_len_ratio"] = data["respa_len"] / data["prompt_len"]  # 応答Aとプロンプトの長さの比率
        data["respb_prompt_len_ratio"] = data["respb_len"] / data["prompt_len"]  # 応答Bとプロンプトの長さの比率
        data["respa_respb_len_ratio"] = data["respa_len"] / data["respb_len"]  # 応答Aと応答Bの長さの比率
        
        data["respa_respb_len_diff"] = data["respa_len"] - data["respb_len"]  # 長さの差
        data["respa_prompt_len_diff"] = data["respa_len"] - data["prompt_len"]  # 応答Aとプロンプトの長さの差
        data["respb_prompt_len_diff"] = data["respb_len"] - data["prompt_len"]  # 応答Bとプロンプトの長さの差
        
        # n-gramオーバーラップの比率を計算
        data["respa_prompt_overlap_unigram_ratio"] = data["respa_prompt_overlap_unigram"] / data["prompt_len"]
        data["respa_prompt_overlap_bigram_ratio"] = data["respa_prompt_overlap_bigram"] / data["prompt_len"]
        data["respa_prompt_overlap_trigram_ratio"] = data["respa_prompt_overlap_trigram"] / data["prompt_len"]

        data["respb_prompt_overlap_unigram_ratio"] = data["respb_prompt_overlap_unigram"] / data["prompt_len"]
        data["respb_prompt_overlap_bigram_ratio"] = data["respb_prompt_overlap_bigram"] / data["prompt_len"]
        data["respb_prompt_overlap_trigram_ratio"] = data["respb_prompt_overlap_trigram"] / data["prompt_len"]
        
        # 引用のカウントを計算
        data["respa_quotes"] = data["response_a"].apply(lambda x: self.count_quotes(x))  # 応答Aの引用数
        data["respb_quotes"] = data["response_b"].apply(lambda x: self.count_quotes(x))  # 応答Bの引用数
        data["prompt_quotes"] = data["prompt"].apply(lambda x: self.count_quotes(x))  # プロンプトの引用数
        
        # コサイン類似度を計算
        data["respa_respb_cosine_sim"] = data.apply(lambda x: self.cosine_sim(x["response_a"], x["response_b"]), axis=1)
        data["respa_respb_jaccard_sim"] = data.apply(lambda x: self.jaccard_sim(x["response_a"], x["response_b"]), axis=1)
        
        data["respa_prompt_cosine_sim"] = data.apply(lambda x: self.cosine_sim(x["response_a"], x["prompt"]), axis=1)
        data["respa_prompt_jaccard_sim"] = data.apply(lambda x: self.jaccard_sim(x["response_a"], x["prompt"]), axis=1)
        
        data["respb_prompt_cosine_sim"] = data.apply(lambda x: self.cosine_sim(x["response_b"], x["prompt"]), axis=1)
        data["respb_prompt_jaccard_sim"] = data.apply(lambda x: self.jaccard_sim(x["response_b"], x["prompt"]), axis=1)
        
        return data  # 処理後のデータを返す
```

</div>
</details>

In [ ]:
class Preprocessor:

    def cosine_sim(self, text1: str, text2: str):
        try:
            vectorizer = TfidfVectorizer().fit_transform([text1, text2])  # テキストのTF-IDFベクトルを作成
            vectors = vectorizer.toarray()  # ベクトルを配列に変換
            cos_sim = cosine_similarity(vectors)  # コサイン類似度を計算
            return cos_sim[0][1]  # コサイン類似度の結果を返す
        except:
            return np.nan  # エラーが発生した場合はNaNを返す

    def jaccard_sim(self, text1: str, text2: str):
        set1 = set(text1.split())  # テキスト1を単語の集合に変換
        set2 = set(text2.split())  # テキスト2を単語の集合に変換
        intersection = set1.intersection(set2)  # 交差部分を計算
        union = set1.union(set2)  # 合併部分を計算
        return len(intersection) / len(union)  # ジャッカード類似度を返す
    
    def count_quotes(self, text: str) -> int:
        single_quote_pattern = r"'(.*?)'"  # 単一引用のパターン
        double_quote_pattern = r'"(.*?)"'  # 二重引用のパターン
        single_quotes = re.findall(single_quote_pattern, text)  # 単一引用を抽出
        double_quotes = re.findall(double_quote_pattern, text)  # 二重引用を抽出
        total_quotes = len(single_quotes) + len(double_quotes)  # 合計の引用数
        return total_quotes  # 合計の引用数を返す

    def tokenize(self, text: str):
        return nltk.word_tokenize(text.lower())  # テキストを小文字にしトークン化

    def generate_ngrams(self, text: str, n: int):
        tokens = self.tokenize(text)  # トークン化されたテキストを取得
        return list(ngrams(tokens, n))  # n-gramを生成

    def count_ngram_overlaps(self, text1: str, text2: str, n: int) -> int:
        try:
            ngrams1 = self.generate_ngrams(text1, n)  # テキスト1からn-gramを生成
            ngrams2 = self.generate_ngrams(text2, n)  # テキスト2からn-gramを生成
            counter1 = Counter(ngrams1)  # テキスト1のn-gramのカウント
            counter2 = Counter(ngrams2)  # テキスト2のn-gramのカウント
            overlap = counter1 & counter2  # 共通のn-gramを計算
            overlap_count = sum(overlap.values())  # 重複の合計を計算
            return overlap_count  # 重複数を返す
        except:
            return 0  # エラーが発生した場合は0を返す
        
    def run(self, data: pd.DataFrame) -> pd.DataFrame:
        
        # それぞれの応答間でのn-gramオーバーラップを計算
        data["respa_respb_overlap_unigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["response_b"], 1), axis=1)
        data["respa_respb_overlap_bigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["response_b"], 2), axis=1)
        data["respa_respb_overlap_trigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["response_b"], 3), axis=1)

        # 各応答とプロンプト間でのn-gramオーバーラップを計算
        data["respa_prompt_overlap_unigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["prompt"], 1), axis=1)
        data["respa_prompt_overlap_bigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["prompt"], 2), axis=1)
        data["respa_prompt_overlap_trigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_a"], x["prompt"], 3), axis=1)

        data["respb_prompt_overlap_unigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_b"], x["prompt"], 1), axis=1)
        data["respb_prompt_overlap_bigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_b"], x["prompt"], 2), axis=1)
        data["respb_prompt_overlap_trigram"] = data.apply(lambda x: self.count_ngram_overlaps(x["response_b"], x["prompt"], 3), axis=1)
        
        # 各応答とプロンプトの長さを計算
        data["respa_len"] = data["response_a"].apply(lambda x: len(self.tokenize(x)))  # 応答Aの長さ
        data["respb_len"] = data["response_b"].apply(lambda x: len(self.tokenize(x)))  # 応答Bの長さ
        data["prompt_len"] = data["prompt"].apply(lambda x: len(self.tokenize(x)))  # プロンプトの長さ
        
        # 異なる長さの比率や差を計算
        data["respa_prompt_len_ratio"] = data["respa_len"] / data["prompt_len"]  # 応答Aとプロンプトの長さの比率
        data["respb_prompt_len_ratio"] = data["respb_len"] / data["prompt_len"]  # 応答Bとプロンプトの長さの比率
        data["respa_respb_len_ratio"] = data["respa_len"] / data["respb_len"]  # 応答Aと応答Bの長さの比率
        
        data["respa_respb_len_diff"] = data["respa_len"] - data["respb_len"]  # 長さの差
        data["respa_prompt_len_diff"] = data["respa_len"] - data["prompt_len"]  # 応答Aとプロンプトの長さの差
        data["respb_prompt_len_diff"] = data["respb_len"] - data["prompt_len"]  # 応答Bとプロンプトの長さの差
        
        # n-gramオーバーラップの比率を計算
        data["respa_prompt_overlap_unigram_ratio"] = data["respa_prompt_overlap_unigram"] / data["prompt_len"]
        data["respa_prompt_overlap_bigram_ratio"] = data["respa_prompt_overlap_bigram"] / data["prompt_len"]
        data["respa_prompt_overlap_trigram_ratio"] = data["respa_prompt_overlap_trigram"] / data["prompt_len"]

        data["respb_prompt_overlap_unigram_ratio"] = data["respb_prompt_overlap_unigram"] / data["prompt_len"]
        data["respb_prompt_overlap_bigram_ratio"] = data["respb_prompt_overlap_bigram"] / data["prompt_len"]
        data["respb_prompt_overlap_trigram_ratio"] = data["respb_prompt_overlap_trigram"] / data["prompt_len"]
        
        # 引用のカウントを計算
        data["respa_quotes"] = data["response_a"].apply(lambda x: self.count_quotes(x))  # 応答Aの引用数
        data["respb_quotes"] = data["response_b"].apply(lambda x: self.count_quotes(x))  # 応答Bの引用数
        data["prompt_quotes"] = data["prompt"].apply(lambda x: self.count_quotes(x))  # プロンプトの引用数
        
        # コサイン類似度を計算
        data["respa_respb_cosine_sim"] = data.apply(lambda x: self.cosine_sim(x["response_a"], x["response_b"]), axis=1)
        data["respa_respb_jaccard_sim"] = data.apply(lambda x: self.jaccard_sim(x["response_a"], x["response_b"]), axis=1)
        
        data["respa_prompt_cosine_sim"] = data.apply(lambda x: self.cosine_sim(x["response_a"], x["prompt"]), axis=1)
        data["respa_prompt_jaccard_sim"] = data.apply(lambda x: self.jaccard_sim(x["response_a"], x["prompt"]), axis=1)
        
        data["respb_prompt_cosine_sim"] = data.apply(lambda x: self.cosine_sim(x["response_b"], x["prompt"]), axis=1)
        data["respb_prompt_jaccard_sim"] = data.apply(lambda x: self.jaccard_sim(x["response_b"], x["prompt"]), axis=1)
        
        return data  # 処理後のデータを返す

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
%%time

preprocessor = Preprocessor()
train = preprocessor.run(train)
test = preprocessor.run(test)
train.head()
```

</div>
<div class="column-right">

# 日本語訳

```python
%%time

preprocessor = Preprocessor()  # Preprocessorクラスのインスタンスを作成
train = preprocessor.run(train)  # トレーニングデータに対して前処理を実行
test = preprocessor.run(test)  # テストデータに対して前処理を実行
train.head()  # トレーニングデータの最初の5行を表示
```

</div>
</details>

In [ ]:
%%time

preprocessor = Preprocessor()  # Preprocessorクラスのインスタンスを作成
train = preprocessor.run(train)  # トレーニングデータに対して前処理を実行
test = preprocessor.run(test)  # テストデータに対して前処理を実行
train.head()  # トレーニングデータの最初の5行を表示

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

### Data Preparation


</div>
<div class="column-right">

# 日本語訳

### データ準備



</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
drop_cols = ["id", "response_a", "response_b", "prompt"]
target_cols = ["winner_model_a", "winner_model_b", "winner_tie"]
target = "target"

train[target] = np.nan
for idx, t in enumerate(target_cols):
    train.loc[train[t] == 1, target] = idx
train[target] = train[target].astype("int32")
    
train.head()
```

</div>
<div class="column-right">

# 日本語訳

```python
drop_cols = ["id", "response_a", "response_b", "prompt"]  # 削除する列のリスト
target_cols = ["winner_model_a", "winner_model_b", "winner_tie"]  # ターゲット列のリスト
target = "target"  # ターゲット変数

train[target] = np.nan  # ターゲット変数の初期化
for idx, t in enumerate(target_cols):
    train.loc[train[t] == 1, target] = idx  # ターゲット列に基づいてインデックスを設定
train[target] = train[target].astype("int32")  # ターゲット変数を整数型に変換
    
train.head()  # トレーニングデータの最初の5行を表示
```

</div>
</details>

In [ ]:
drop_cols = ["id", "response_a", "response_b", "prompt"]  # 削除する列のリスト
target_cols = ["winner_model_a", "winner_model_b", "winner_tie"]  # ターゲット列のリスト
target = "target"  # ターゲット変数

train[target] = np.nan  # ターゲット変数の初期化
for idx, t in enumerate(target_cols):
    train.loc[train[t] == 1, target] = idx  # ターゲット列に基づいてインデックスを設定
train[target] = train[target].astype("int32")  # ターゲット変数を整数型に変換
    
train.head()  # トレーニングデータの最初の5行を表示

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
X = train.drop(columns=target_cols+drop_cols+[target]+["model_a", "model_b"], axis=1)
y = train[target]
X_test = test.drop(columns=drop_cols, axis=1)

X = X.replace([-np.inf, np.inf], np.nan)
X_test = X_test.replace([-np.inf, np.inf], np.nan)
```

</div>
<div class="column-right">

# 日本語訳

```python
X = train.drop(columns=target_cols+drop_cols+[target]+["model_a", "model_b"], axis=1)  # 特徴量マトリックスXを作成
y = train[target]  # ターゲット変数yを設定
X_test = test.drop(columns=drop_cols, axis=1)  # テストデータの特徴量マトリックスX_testを作成

# 無限大や負無限大をNaNに置き換える
X = X.replace([-np.inf, np.inf], np.nan)
X_test = X_test.replace([-np.inf, np.inf], np.nan)
```

</div>
</details>

In [ ]:
X = train.drop(columns=target_cols+drop_cols+[target]+["model_a", "model_b"], axis=1)  # 特徴量マトリックスXを作成
y = train[target]  # ターゲット変数yを設定
X_test = test.drop(columns=drop_cols, axis=1)  # テストデータの特徴量マトリックスX_testを作成

# 無限大や負無限大をNaNに置き換える
X = X.replace([-np.inf, np.inf], np.nan)
X_test = X_test.replace([-np.inf, np.inf], np.nan)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Handle missing values
imputer = SimpleImputer(strategy='mean')
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)
X_test = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)
```

</div>
<div class="column-right">

# 日本語訳

```python
# 欠損値の処理
imputer = SimpleImputer(strategy='mean')  # 平均で欠損値を補完するためのインプッターを作成
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)  # トレーニングデータの欠損値を補完
X_test = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)  # テストデータの欠損値を補完
```

</div>
</details>

In [ ]:
# 欠損値の処理
imputer = SimpleImputer(strategy='mean')  # 平均で欠損値を補完するためのインプッターを作成
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)  # トレーニングデータの欠損値を補完
X_test = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)  # テストデータの欠損値を補完

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Feature Selection
selector = SelectKBest(f_classif, k=25)
X_new = selector.fit_transform(X, y)
X_test_new = selector.transform(X_test)
```

</div>
<div class="column-right">

# 日本語訳

```python
# 特徴選択
selector = SelectKBest(f_classif, k=25)  # 最良のk個の特徴量を選択
X_new = selector.fit_transform(X, y)  # トレーニングデータに対して特徴選択を適用
X_test_new = selector.transform(X_test)  # テストデータに対して同様に適用
```

</div>
</details>

In [ ]:
# 特徴選択
selector = SelectKBest(f_classif, k=25)  # 最良のk個の特徴量を選択
X_new = selector.fit_transform(X, y)  # トレーニングデータに対して特徴選択を適用
X_test_new = selector.transform(X_test)  # テストデータに対して同様に適用

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# 🧩 Model Training and Evaluation

### Model Definitions

Defines several machine learning models including Random Forest, Gradient Boosting, SVM, XGBoost, CatBoost, and a Voting Classifier.

### Feature Selection

Uses SelectKBest to select 25 best features based on ANOVA F-value between feature and target.

### Cross-validation

Uses Stratified K-Fold cross-validation for model evaluation.

### Training and Evaluation

Iterates through each model, trains it using the training data, evaluates using cross-validation, and calculates the mean CV Log Loss.

### Feature Importances

Calculates and stores feature importances for applicable models (Random Forest, Gradient Boosting, XGBoost, CatBoost).

### Best Model Identification

Identifies the best performing model based on the lowest CV Log Loss.

### Results Display

Displays the results in a DataFrame showing the CV Log Loss for each model and, if applicable, feature importances.


</div>
<div class="column-right">

# 日本語訳

# 🧩 モデルのトレーニングと評価

### モデルの定義

ランダムフォレスト、勾配ブースティング、SVM、XGBoost、CatBoost、および投票分類器を含むいくつかの機械学習モデルを定義します。

### 特徴選択

ANOVA F値に基づいて、SelectKBestを使用して25の最良特徴を選択します。

### クロスバリデーション

モデル評価のために層化K分割交差検証を使用します。

### トレーニングと評価

各モデルを反復処理し、トレーニングデータを使ってトレーニングし、クロスバリデーションを使用して評価し、平均CVログロスを計算します。

### 特徴の重要度

適用可能なモデル（ランダムフォレスト、勾配ブースティング、XGBoost、CatBoost）について、特徴の重要度を計算して保存します。

### 最良モデルの特定

最も低いCVログロスに基づいて、最も良いパフォーマンスのモデルを特定します。

### 結果の表示

各モデルのCVログロスを示し、適用可能な場合は特徴の重要度を表示するDataFrameを表示します。



</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python

# Define models and their configurations
models = {
    'random_forest': {
        'model': RandomForestClassifier(
            n_estimators=100,
            max_depth=10,
            random_state=config.seed
        ),
        'params': {}
    },
    'gradient_boosting': {
        'model': GradientBoostingClassifier(
            n_estimators=300,
            learning_rate=0.1,
            max_depth=5,
            subsample=0.8,
            random_state=config.seed
        ),
        'params': {}
    },
    'svm': {
        'model': SVC(
            kernel='rbf',
            C=1.0,
            gamma='scale',
            probability=True,
            random_state=config.seed
        ),
        'params': {}
    },
    'xgboost': {
        'model': xgb.XGBClassifier(
            objective='multi:softprob',
            num_class=3,
            eval_metric='mlogloss',
            subsample=0.8,
            n_estimators=650,
            learning_rate=0.045,
            max_depth=5,
            random_state=config.seed,
#             tree_method='gpu_hist'  # GPU acceleration if available
        ),
        'params': {}
    },
    'catboost': {
        'model': cb.CatBoostClassifier(
            loss_function='MultiClass',
            iterations=650,
            learning_rate=0.045,
            depth=5,
            random_seed=config.seed,
#             task_type="GPU",  # Use GPU if available
            verbose=75
        ),
        'params': {}
    },
    'voting': {
        'model': VotingClassifier(
            estimators=[
                ('lr', LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=200)),
                ('svc', SVC(probability=True)),
                ('knn', KNeighborsClassifier(n_neighbors=5))
            ],
            voting='soft'
        ),
        'params': {}
    }
}

# Select features using SelectKBest
selector = SelectKBest(f_classif, k=25)
X_new = selector.fit_transform(X, y)
X_test_new = selector.transform(X_test)

# Cross-validation setup
cv = StratifiedKFold(n_splits=config.n_splits, shuffle=True, random_state=config.seed)

# Dataframe to store results
results = []

# Iterate over models
for model_name, model_data in models.items():
    model = model_data['model']
    print(f"Training model: {model_name}")

    test_preds = np.zeros(shape=(X_test_new.shape[0], y.nunique()))
    cv_scores = []

    for idx, (train_idx, val_idx) in enumerate(cv.split(X_new, y)):
        X_train, y_train = X_new[train_idx], y[train_idx]
        X_val, y_val = X_new[val_idx], y[val_idx]

        if model_name == 'voting':
            model.fit(X_train, y_train)
        elif model_name == 'catboost':
            model.fit(
                X_train,
                y_train,
                eval_set=[(X_train, y_train), (X_val, y_val)],
                early_stopping_rounds=75,
                verbose=75
            )
        else:
            model.fit(
                X_train,
                y_train
            )

        if model_name != 'voting':
            val_preds = model.predict_proba(X_val)
            val_log_loss = log_loss(y_val, val_preds, eps="auto")
            cv_scores.append(val_log_loss)

            test_preds += model.predict_proba(X_test_new) / cv.get_n_splits()

    if model_name != 'voting':
        mean_cv_log_loss = np.mean(cv_scores)
        results.append({'Model': model_name, 'CV_Log_Loss': mean_cv_log_loss})
        print(f"Mean CV Log Loss: {mean_cv_log_loss:.5f}")

# Store feature importances if applicable
if model_name in ['random_forest', 'gradient_boosting', 'xgboost', 'catboost']:
    features = X.columns[selector.get_support()].tolist()
    feat_imp_df = pd.DataFrame({"feature": features})
    feat_imp_df[f"{model_name}_avg_importance"] = 0

    for idx, (_, val_idx) in enumerate(cv.split(X_new, y)):
        X_val, _ = X_new[val_idx], y[val_idx]
        feat_imp_df[f"{model_name}_avg_importance"] += model.feature_importances_ / cv.get_n_splits()

    results_df = pd.DataFrame(results)
    results_df = pd.concat([results_df, feat_imp_df], axis=1)

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Identify the best model
best_model = results_df.loc[results_df['CV_Log_Loss'].idxmin()]
print(f"\nBest Model:\n{best_model}")

# Display results DataFrame
print("\nResults DataFrame:")
print(results_df)

```

</div>
<div class="column-right">

# 日本語訳

```python
# モデルとその設定を定義
models = {
    'random_forest': {
        'model': RandomForestClassifier(
            n_estimators=100,
            max_depth=10,
            random_state=config.seed
        ),
        'params': {}
    },
    'gradient_boosting': {
        'model': GradientBoostingClassifier(
            n_estimators=300,
            learning_rate=0.1,
            max_depth=5,
            subsample=0.8,
            random_state=config.seed
        ),
        'params': {}
    },
    'svm': {
        'model': SVC(
            kernel='rbf',
            C=1.0,
            gamma='scale',
            probability=True,
            random_state=config.seed
        ),
        'params': {}
    },
    'xgboost': {
        'model': xgb.XGBClassifier(
            objective='multi:softprob',
            num_class=3,
            eval_metric='mlogloss',
            subsample=0.8,
            n_estimators=650,
            learning_rate=0.045,
            max_depth=5,
            random_state=config.seed,
#             tree_method='gpu_hist'  # グラフィックデバイスが利用可能ならばGPU加速を使用
        ),
        'params': {}
    },
    'catboost': {
        'model': cb.CatBoostClassifier(
            loss_function='MultiClass',
            iterations=650,
            learning_rate=0.045,
            depth=5,
            random_seed=config.seed,
#             task_type="GPU",  # GPUが利用可能ならば使用
            verbose=75
        ),
        'params': {}
    },
    'voting': {
        'model': VotingClassifier(
            estimators=[
                ('lr', LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=200)),
                ('svc', SVC(probability=True)),
                ('knn', KNeighborsClassifier(n_neighbors=5))
            ],
            voting='soft'
        ),
        'params': {}
    }
}

# SelectKBestを使用して特徴を選択
selector = SelectKBest(f_classif, k=25)
X_new = selector.fit_transform(X, y)  # 特徴量マトリックスXを新しい特徴に変換
X_test_new = selector.transform(X_test)  # テストデータに対しても新しい特徴を適用

# クロスバリデーションの設定
cv = StratifiedKFold(n_splits=config.n_splits, shuffle=True, random_state=config.seed)

# 結果を格納するDataFrame
results = []

# モデルを反復処理
for model_name, model_data in models.items():
    model = model_data['model']
    print(f"モデルのトレーニング: {model_name}")

    test_preds = np.zeros(shape=(X_test_new.shape[0], y.nunique()))  # テスト予測の配列を初期化
    cv_scores = []  # CVスコアを格納するリスト

    for idx, (train_idx, val_idx) in enumerate(cv.split(X_new, y)):
        X_train, y_train = X_new[train_idx], y[train_idx]  # トレーニングデータとラベル
        X_val, y_val = X_new[val_idx], y[val_idx]  # 検証データとラベル

        if model_name == 'voting':
            model.fit(X_train, y_train)  # 投票モデルをトレーニング
        elif model_name == 'catboost':
            model.fit(
                X_train,
                y_train,
                eval_set=[(X_train, y_train), (X_val, y_val)],
                early_stopping_rounds=75,  # 早期停止
                verbose=75
            )
        else:
            model.fit(X_train, y_train)  # その他のモデルをトレーニング

        # 検証データで予測を行う
        if model_name != 'voting':
            val_preds = model.predict_proba(X_val)  # 検証データの確率予測
            val_log_loss = log_loss(y_val, val_preds, eps="auto")  # ログロスを計算
            cv_scores.append(val_log_loss)  # スコアをリストに追加

            test_preds += model.predict_proba(X_test_new) / cv.get_n_splits()  # テスト予測を集積

    if model_name != 'voting':
        mean_cv_log_loss = np.mean(cv_scores)  # 平均CVログロスを計算
        results.append({'Model': model_name, 'CV_Log_Loss': mean_cv_log_loss})  # 結果を格納
        print(f"平均CVログロス: {mean_cv_log_loss:.5f}")  # 結果を表示

# 特徴重要度を格納（適用可能なモデルに対して）
if model_name in ['random_forest', 'gradient_boosting', 'xgboost', 'catboost']:
    features = X.columns[selector.get_support()].tolist()  # 選択された特徴
    feat_imp_df = pd.DataFrame({"feature": features})  # 特徴重要度のDataFrameを作成
    feat_imp_df[f"{model_name}_avg_importance"] = 0  # 初期化

    for idx, (_, val_idx) in enumerate(cv.split(X_new, y)):
        X_val, _ = X_new[val_idx], y[val_idx]  # 検証データ
        feat_imp_df[f"{model_name}_avg_importance"] += model.feature_importances_ / cv.get_n_splits()  # 重みを集計

    results_df = pd.DataFrame(results)  # 結果のDataFrameを作成
    results_df = pd.concat([results_df, feat_imp_df], axis=1)  # 特徴重要度を結合

# 結果をDataFrameに変換
results_df = pd.DataFrame(results)

# 最良モデルの特定
best_model = results_df.loc[results_df['CV_Log_Loss'].idxmin()]  # 最小のCVログロスを持つモデルを特定
print(f"\n最良モデル:\n{best_model}")

# 結果のDataFrameを表示
print("\n結果のDataFrame:")
print(results_df)
```

</div>
</details>

In [ ]:
# モデルとその設定を定義
models = {
    'random_forest': {
        'model': RandomForestClassifier(
            n_estimators=100,
            max_depth=10,
            random_state=config.seed
        ),
        'params': {}
    },
    'gradient_boosting': {
        'model': GradientBoostingClassifier(
            n_estimators=300,
            learning_rate=0.1,
            max_depth=5,
            subsample=0.8,
            random_state=config.seed
        ),
        'params': {}
    },
    'svm': {
        'model': SVC(
            kernel='rbf',
            C=1.0,
            gamma='scale',
            probability=True,
            random_state=config.seed
        ),
        'params': {}
    },
    'xgboost': {
        'model': xgb.XGBClassifier(
            objective='multi:softprob',
            num_class=3,
            eval_metric='mlogloss',
            subsample=0.8,
            n_estimators=650,
            learning_rate=0.045,
            max_depth=5,
            random_state=config.seed,
#             tree_method='gpu_hist'  # グラフィックデバイスが利用可能ならばGPU加速を使用
        ),
        'params': {}
    },
    'catboost': {
        'model': cb.CatBoostClassifier(
            loss_function='MultiClass',
            iterations=650,
            learning_rate=0.045,
            depth=5,
            random_seed=config.seed,
#             task_type="GPU",  # GPUが利用可能ならば使用
            verbose=75
        ),
        'params': {}
    },
    'voting': {
        'model': VotingClassifier(
            estimators=[
                ('lr', LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=200)),
                ('svc', SVC(probability=True)),
                ('knn', KNeighborsClassifier(n_neighbors=5))
            ],
            voting='soft'
        ),
        'params': {}
    }
}

# SelectKBestを使用して特徴を選択
selector = SelectKBest(f_classif, k=25)
X_new = selector.fit_transform(X, y)  # 特徴量マトリックスXを新しい特徴に変換
X_test_new = selector.transform(X_test)  # テストデータに対しても新しい特徴を適用

# クロスバリデーションの設定
cv = StratifiedKFold(n_splits=config.n_splits, shuffle=True, random_state=config.seed)

# 結果を格納するDataFrame
results = []

# モデルを反復処理
for model_name, model_data in models.items():
    model = model_data['model']
    print(f"モデルのトレーニング: {model_name}")

    test_preds = np.zeros(shape=(X_test_new.shape[0], y.nunique()))  # テスト予測の配列を初期化
    cv_scores = []  # CVスコアを格納するリスト

    for idx, (train_idx, val_idx) in enumerate(cv.split(X_new, y)):
        X_train, y_train = X_new[train_idx], y[train_idx]  # トレーニングデータとラベル
        X_val, y_val = X_new[val_idx], y[val_idx]  # 検証データとラベル

        if model_name == 'voting':
            model.fit(X_train, y_train)  # 投票モデルをトレーニング
        elif model_name == 'catboost':
            model.fit(
                X_train,
                y_train,
                eval_set=[(X_train, y_train), (X_val, y_val)],
                early_stopping_rounds=75,  # 早期停止
                verbose=75
            )
        else:
            model.fit(X_train, y_train)  # その他のモデルをトレーニング

        # 検証データで予測を行う
        if model_name != 'voting':
            val_preds = model.predict_proba(X_val)  # 検証データの確率予測
            val_log_loss = log_loss(y_val, val_preds, eps="auto")  # ログロスを計算
            cv_scores.append(val_log_loss)  # スコアをリストに追加

            test_preds += model.predict_proba(X_test_new) / cv.get_n_splits()  # テスト予測を集積

    if model_name != 'voting':
        mean_cv_log_loss = np.mean(cv_scores)  # 平均CVログロスを計算
        results.append({'Model': model_name, 'CV_Log_Loss': mean_cv_log_loss})  # 結果を格納
        print(f"平均CVログロス: {mean_cv_log_loss:.5f}")  # 結果を表示

# 特徴重要度を格納（適用可能なモデルに対して）
if model_name in ['random_forest', 'gradient_boosting', 'xgboost', 'catboost']:
    features = X.columns[selector.get_support()].tolist()  # 選択された特徴
    feat_imp_df = pd.DataFrame({"feature": features})  # 特徴重要度のDataFrameを作成
    feat_imp_df[f"{model_name}_avg_importance"] = 0  # 初期化

    for idx, (_, val_idx) in enumerate(cv.split(X_new, y)):
        X_val, _ = X_new[val_idx], y[val_idx]  # 検証データ
        feat_imp_df[f"{model_name}_avg_importance"] += model.feature_importances_ / cv.get_n_splits()  # 重みを集計

    results_df = pd.DataFrame(results)  # 結果のDataFrameを作成
    results_df = pd.concat([results_df, feat_imp_df], axis=1)  # 特徴重要度を結合

# 結果をDataFrameに変換
results_df = pd.DataFrame(results)

# 最良モデルの特定
best_model = results_df.loc[results_df['CV_Log_Loss'].idxmin()]  # 最小のCVログロスを持つモデルを特定
print(f"\n最良モデル:\n{best_model}")

# 結果のDataFrameを表示
print("\n結果のDataFrame:")
print(results_df)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
for idx, t in enumerate(target_cols):
    sample_submission[t] = test_preds[:, idx]
sample_submission.head()
```

</div>
<div class="column-right">

# 日本語訳

```python
for idx, t in enumerate(target_cols):
    sample_submission[t] = test_preds[:, idx]  # 各ターゲット列にテスト予測を格納
sample_submission.head()  # 提出用DataFrameの最初の5行を表示
```

</div>
</details>

In [ ]:
for idx, t in enumerate(target_cols):
    sample_submission[t] = test_preds[:, idx]  # 各ターゲット列にテスト予測を格納
sample_submission.head()  # 提出用DataFrameの最初の5行を表示

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
sample_submission.to_csv("submission.csv", index=False)
```

</div>
<div class="column-right">

# 日本語訳

```python
sample_submission.to_csv("submission.csv", index=False)  # 提出ファイルをCSVとして保存
```

</div>
</details>

In [ ]:
sample_submission.to_csv("submission.csv", index=False)  # 提出ファイルをCSVとして保存